<a href="https://colab.research.google.com/github/lnunezma/Rentavariableyderivados22/blob/main/derivados02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calcular el precio de un bono
Puede consultar el archivo de Excel: [excel_para_pyhton.xlsm](https://www.dropbox.com/s/wzq2y2k97c1d6rg/excel_para_pyhton.xlsm?dl=1)

## Precio utilizando la TIR
Puede ver la **Hoja1** del documento de Excel.


Supongamos un bono con las siguientes características:
* Nominal 1.000 €
* Cupón anual 10%
* Madura al quinto año
* TIR: r = 8%

Se pide:
* Calcular el precio del bono
* Una vez conocido el precio del bono, comprobar que la TIR coincide con el dato proporcionado



In [1]:
pip install numpy-financial

In [ ]:
import numpy_financial as npf

# DATOS DEL BONO
nominal = 1000
cupon = 0.1
n = 5  # años
r = 0.08

# FLUJOS DE CAJA
flujos = [0]
for i in range(1, n+1):
    if i < n:
        flujos.append(cupon * nominal)
    elif i == n:
        flujos.append(cupon * nominal + nominal)
print("Flujos de caja: ", flujos)

# PRECIO DEL BONO
precio = 0
for i in range(1, n+1):
    precio += flujos[i] / (1+r)**i
print("Precio del bono:", precio)
bono = flujos[:]
bono[0] = -precio

# COMPROBACIÓN TIR = r
tir = npf.irr(bono)
print(f"La TIR del bono es: {tir:.5%}")  # Internal rate of return

## Precio utilizando la ETTI
Puede ver la **Hoja2** del documento de Excel.


Supongamos un bono con las siguientes características:
* Nominal 1.000 €
* Cupón anual 10%
* Madura al quinto año
* La ETTI para los diferentes años es:
 - año 1: 2%
 - año 2: 4%
 - año 3: 6%
 - año 4: 8%
 - año 5: 10%

Se pide:
* Calcular el precio del bono
* Calcular la TIR

In [ ]:
import numpy_financial as npf

# DATOS
nominal = 1000
cupon = 0.1
n = 5 # años
r01 = 0.02
r02 = 0.04
r03 = 0.06
r04 = 0.08
r05 = 0.10
etti = [r01,r02,r03,r04,r05]

# FLUJOS DE CAJA
flujos = [0]
for i in range(1, n+1):
    if i < n:
        flujos.append(cupon * nominal)
    elif i == n:
        flujos.append(cupon * nominal + nominal)
print("Flujos de caja: ", flujos)

# PRECIO DEL BONO CON LA ETTI
precio = 0
for i in range(1, n+1):
    precio += flujos[i] / (1+etti[i-1])**i
print("Precio del bono:", precio)
bono = flujos[:]
bono[0] = -precio

# CALCULAR LA TIR DEL BONO TIR = r
tir = npf.irr(bono)
print(f"La TIR del bono es: {tir:.6%}")  # Internal rate of return

# Creación de un bono sintético cupón cero
Puede ver la **Hoja3** del documento de Excel.


Supongamos dos bonos A y B, ambos a 5 años con las siguientes características:
* Bono A
 - Nominal: 10.000 €
 - Cupón anual: 2%
* Bono B
 - Nominal: 4.000 €
 - Cupón anual: 25%
* La ETTI para los diferentes años es:
 - año 1: 2%
 - año 2: 4%
 - año 3: 6%
 - año 4: 8%
 - año 5: 10%

Se pide:
* Calcular el precio de ambos bonos
* Calcular la TIR de ambos bonos
* Crear el bono C que es un bono sintético que se forma combinando los bonos A y B para conseguir un bono cupón cero a 5 años.
* Calcular la TIR del bono C y comprobar que es igual a la ETTI a 5 años, que es del 10%.

In [ ]:
import numpy_financial as npf

# DATOS BONO A
nominalA = 10_000
cuponA = 0.02
n = 5 # años, para A y B
# DATOS BONO B
nominalB = 4_000
cuponB = 0.25
# ETTI
etti =[.02,.04,.06,.08,.10]

# FLUJOS DE CAJA DE LOS BONOS
def flujos(nominal, cupon, n):
    cf = [0]  # array con el cash flow
    for i in range(1, n+1):
        if i < n:
            cf.append(cupon * nominal)
        elif i == n:
            cf.append(cupon * nominal + nominal)
    return cf

# Flujos de caja de los Bonos A y B
flujosA = flujos(nominalA, cuponA, n)
flujosB = flujos(nominalB, cuponB, n)

# PRECIO DE UN BONO CON LA ETTI
def precio_etti(flujos, etti):
    precio = 0
    for i in range(1, n+1):
        precio += flujos[i] * (1+etti[i-1])**-i
    return precio

# Precios de los Bonos A y B
precioA = precio_etti(flujosA, etti)
precioB = precio_etti(flujosB, etti)
print(f"El precio del bono A es {precioA}")
print(f"El precio del bono B es {precioB}")
# Flujos de caja de los bonos A y B
bonoA = flujosA[:]
bonoA[0] = -precioA
bonoB = flujosB[:]
bonoB[0] = -precioB

# CREACIÓN DEL BONO SINTÉTICO C

# m es el número de bonos que se han de comprar o vender de uno de los bonos
m = max(cuponA*nominalA, cuponB*nominalB) / min(cuponB*nominalB, cuponA*nominalA)
# con los datos de ejemplo m = 1000 / 200 = 5

# Flujos de caja del bono C
bonoC = [0]*(n+1)
for i in range(0, n+1):
    bonoC[i] = m * bonoA[i] - bonoB[i]
print(f"Flujos de caja del bono C: {bonoC}")

# Calcular la TIR de los bonos
tirA = npf.irr(bonoA)
tirB = npf.irr(bonoB)
tirC = npf.irr(bonoC)
print(f"La TIR del bono A es: {tirA:.6%}")
print(f"La TIR del bono B es: {tirB:.6%}")
print(f"La TIR del bono C es: {tirC:.6%}")

# Reinversión de flujos de caja intermedios
Puede ver la **Hoja4** del documento de Excel.


La importancia de trabajar con bonos cupón cero.

La TIR es la Tasa Interna de Rentabilidad, la palabra 'Interna' indica que si la operación financiera analizada se mezcla con otras operaciones la rentabilidad prometida por la TIR se podría alterar.

Una operación de inversión que tenga flujos de caja intermedios, por ejemplo un bono cupón explícito, puede no proporcionar a su propietario la rentabilidad que promete la TIR del bono si el inversor no se preocupa de reinvertir los flujos de caja intermedios hasta el final de la operación.

Supongamos un bono cupón explícito con las siguientes características:
* Nominal 10.000 €
* Cupón 10%
* Madura a los 10 años
* Precio de adquisición 10.000 €

Se pide:
* Calcular la TIR del bono
* Calcular la rentabilidad del inversor supueto que reinvierta los flujos de caja intermedios hasta el momento de vencimiento, a las siguientes rentabilidades:
 - al 0%
 - al 10%
 - al 20%
* Crear un bono cupón cero, al mismo plazo, por el mismo precio, cuyo último flujo de caja sea el necesario para proporcionar una rentabilidad del 10% y comprobar que su TIR es de 10%.

In [ ]:
import numpy_financial as npf

# DATOS
nominal = 10_000
cupon = .1
n = 10 # años
precio = 10_000

# Flujos de caja del bono
cf = [0]*(n+1)
for i in range(1,n+1):
    cf[i] = cupon * nominal
cf[n] += nominal
cf[0] = -precio

print(f"La TIR del bono es: {npf.irr(cf):.2%}")

# Montante de la Reinversión
def montante(cf,r):
    m = 0 # montante
    for i in range(1, n+1):
        m += cf[i]*(1+r)**(n-i)
    return m

tasas_reinversion = [0,.1,.2]
for t in tasas_reinversion:
    m = montante(cf,t)
    print()
    print(f"El montante reinvirtiendo al tanto del {t:.0%} es {m:,.2f} €")
    print(f"La rentabilidad del inversor reinvirtiendo al tanto {t:.0%} es {(m/precio)**(1/n)-1:.2%}")

La TIR del bono es: 10.00%

El montante reinvirtiendo al tanto del 0% es 20,000.00 €
La rentabilidad del inversor reinvirtiendo al tanto 0% es 7.18%

El montante reinvirtiendo al tanto del 10% es 25,937.42 €
La rentabilidad del inversor reinvirtiendo al tanto 10% es 10.00%

El montante reinvirtiendo al tanto del 20% es 35,958.68 €
La rentabilidad del inversor reinvirtiendo al tanto 20% es 13.65%


# Comparación entre una inversión cierta y otra aleatoria
Puede ver la **Hoja5** del documento de Excel.


Comparemos dos inversiones A y B, ambas de duración 1 año y que proporcionan ambas un 10% anual, con una inversión de un millón de euros cada una. La inversión A es cierta, esto supone que existe un 100% de probabilidad de que el flujo de caja final sea 1.100.000 €. La inversión B, tiene una probabilidad del 1% de que el montante final sea m1, y una probabilidad del 99% de que el montante final sea m2.  
Se pide:  
* Caso 1: si m1 = 0, calcular m2, para que la rentabilidad esperada de la inversión B sea del 10%
* Caso 2: si m1 = 500.000, calcular m2, para que la rentabildidad esperada de B sea el 10%
* Comprobar que la TIR de ambas inversiones es del 10%

In [ ]:
import numpy_financial as npf

# DATOS
c = 1_000_000 # capital inicial
r = .1 # rentabilidad 10% anual
ma = c*(1+r) # Montante A: el montante de la inversión A es 1.100.000
p = .01 # probabilidad de que suceda una de las ramas de la inverión B
q = 1-p # probabilidad complementaria de la otra rama de B, se cumple que p+q=1

# Cálculo de mb2
# El montante de la inversión B es mb y tiene dos componentes mb1 + mb2 = mb
def mb_2(ma,p,mb1):
    return (ma - p * mb1) / q

# Resultados
print("CASO 1")
mb1 = 500_000 # primer caso que nos piden
mb2 = mb_2(ma,p,mb1)
print(f"Si el montante alcanzado con mb1 es {mb1:,} entonces el montante mb2 es {mb2:,.2f}")
print(f"La TIR de la inversión B1 es: {npf.irr([-c,mb1]):.1%}")
print(f"La TIR de la inversión B2 es: {npf.irr([-c,mb2]):.2%}")
print(f"La TIR de la inversión A es: {npf.irr([-c, ma]):.1%}")          # comprobar que ambas TIR son del 10%
print(f"La TIR de la inversión B es: {npf.irr([-c, p*mb1+q*mb2]):.1%}") # comprobar que ambas TIR son del 10%
print()

print("CASO 2")
mb1 = 0       # segundo caso que nos piden
mb2 = mb_2(ma,p,mb1)
print(f"Si el montante alcanzado con mb1 es {mb1} entonces el montante mb2 es {mb2:,.2f}")
print(f"La TIR de la inversión B1 es: {npf.irr([-c,mb1]):.1%}")
print(f"La TIR de la inversión B2 es: {npf.irr([-c,mb2]):.2%}")
print(f"La TIR de la inversión A es: {npf.irr([-c, ma]):.1%}")          # comprobar que ambas TIR son del 10%
print(f"La TIR de la inversión B es: {npf.irr([-c, p*mb1+q*mb2]):.1%}") # comprobar que ambas TIR son del 10%

CASO 1
Si el montante alcanzado con mb1 es 500,000 entonces el montante mb2 es 1,106,060.61
La TIR de la inversión B1 es: -50.0%
La TIR de la inversión B2 es: 10.61%
La TIR de la inversión A es: 10.0%
La TIR de la inversión B es: 10.0%

CASO 2
Si el montante alcanzado con mb1 es 0 entonces el montante mb2 es 1,111,111.11
La TIR de la inversión B1 es: nan%
La TIR de la inversión B2 es: 11.11%
La TIR de la inversión A es: 10.0%
La TIR de la inversión B es: 10.0%


# Forward $r_{12}$ conocidos dos bonos
Puede ver la **Hoja6** del documento de Excel.


En el mercado cotizan los bonos A y B.
* El bono A es un bono cupón cero de duración un año y TIR del 10%
* El bono B es un bono cupón explícito de 4.400 € anuales,  nominal 10.000 €, duración 2 años y precio de 14.000 €
* El bono C es un bono cupón cero a dos años, que se obtiene como bono sintético combinando los bonos A y B

Se pide:
* Calcular la TIR del bono C
* Comprobar que la TIR del bono C es el punto de la ETTI a dos años ($r_{02}$). Para hacer la comprobación calcular con la ETTI los precios de los bonos A, B y C y comprobar que coinciden con los previstos.
* Comprobar que el nominal elegido para el bono A puede ser cualquiera, es arbitrario.

In [ ]:
import numpy_financial as npf
from random import randint, seed
seed()

# DATOS
tirA = .1
cuponB = 4_400
nominalB = 10_000
precioB = 14_000
nominalA = randint(1,10000)
print(f"El nominal elegido para el bono A es {nominalA}")

# FLUJOS DE CAJA bonos A y B
bonoB = [-precioB, cuponB, cuponB+nominalB]
bonoA = [-nominalA/(1+tirA), nominalA, 0]    # añadimos un flujo 0 en t=2 por comodidad, para luego restar flujos
m = bonoB[1]/bonoA[1]
print(f"El multiplicador m es {m}")
bonoC = [bonoB[0]-m*bonoA[0], bonoB[1]-m*bonoA[1], bonoB[2]-m*bonoA[2]]
print("Bono C: ", bonoC)

# TIR del bono C
tirC = npf.irr(bonoC)
print(f"La TIR del bono C es: {tirC:.6%}")

# Creación de la ETTI
etti = [tirA, tirC] # la tirC si pertenece a la ETTI por ser un bono cupón cero, pero la tirB no pertenece

# COMPROBAR PRECIOS CON LA ETTI
def precio_etti(flujos, etti):
    precio = 0
    for i in range(1, len(flujos)):
        precio += flujos[i] * (1+etti[i-1])**-i
    return precio

# Precios de los Bonos A, B y C
precioA = precio_etti(bonoA, etti)
precioB = precio_etti(bonoB, etti)
precioC = precio_etti(bonoC, etti)
print(f"La diferencia de precios en el bono A es {precioA+bonoA[0]}")
print(f"La diferencia de precios en el bono B es {precioB+bonoB[0]}")
print(f"La diferencia de precios en el bono C es {precioC+bonoC[0]}")

El nominal elegido para el bono A es 4860
El multiplicador m es 0.9053497942386831
Bono C:  [-10000.0, 0.0, 14400.0]
La TIR del bono C es: 20.000000%
La diferencia de precios en el bono A es 0.0
La diferencia de precios en el bono B es -3.637978807091713e-12
La diferencia de precios en el bono C es -3.637978807091713e-12


# Calcular la ETTI con una cartera de tres bonos
Puede ver la **Hoja7** del documento de Excel.

### FASE 1
En el mercado cotizan los bonos A , B y C.
* El bono A es un bono cupón cero a un año de nominal 1.000 € y TIR del 10%
* El bono B es un bono cupón explícito de 90 € anuales,  nominal 1.000 € y duración 2 años
* El bono C es un bono cupón explícito de 500 € anuales, nominal 1.000 € y duración 3 años
* Conocemos la ETTI a 1, 2 y 3 años que es 10%, 20% y 30%, respectivamente  

Se pide:
* Calcular los precios de los bonos A, B y C
* Calcular la TIR de los bonos A, B y C

### FASE 2
Supongamos conocidos los flujos de caja de los bonos A, B y C incluidos sus precios.  

Se pide:
* Calcular la ETTI de los años 1, 2 y 3.

In [2]:
import numpy_financial as npf       #importamos la libreria financiera

#####   FASE 1   #####
# Flujos de caja de los bonos
bonoA = [0,1000,0,0]     # inicialmente el flujo de caja inicial es cero, es recomendable poner 4 espacios en todos para evitar errores
bonoB = [0,90,1090,0]
bonoC = [0,500,500,1500]

# Calcular el precio de los bonos dada la ETTI
etti = [.1, .2, .3]         #esto es una lista y conceptualmente se conoce como array (es un vector)
def precio_etti(flujos, etti):      #creamos la función para calcular el precio (la hemos visto anteriormente)
    precio = 0
    for i in range(1, len(flujos)):
        precio += flujos[i] * (1+etti[i-1])**-i
    return precio

precioA = precio_etti(bonoA, etti)
precioB = precio_etti(bonoB, etti)
precioC = precio_etti(bonoC, etti)
print(f"El precio del bono A es {precioA:,.2f} €")
print(f"El precio del bono B es {precioB:,.2f} €")
print(f"El precio del bono C es {precioC:,.2f} €")
print()

# Calcular la TIR de los bonos
bonoA[0] = -precioA
bonoB[0] = -precioB
bonoC[0] = -precioC

tirA = npf.irr(bonoA)
tirB = npf.irr(bonoB)
tirC = npf.irr(bonoC)

print(f"La TIR del bono A es: {tirA:.2%}")
print(f"La TIR del bono B es: {tirB:.2%}")
print(f"La TIR del bono C es: {tirC:.2%}")
print()

#####   FASE 2   #####
etti_calculada = [0,0,0]  # inicializamos el array (montamos un vector, y como sabemos cuales van a ser ponemos 0, y tres por nº de años)
etti_calculada[0] = tirA  # la ETTI del año 1 es la TIR del bono A por ser un bono cupón cero a un año
def calcula_etti(bono, etti_calculada):     #esta es la misma formula que en excel
    n = [i for i, e in enumerate(bono) if e != 0][-1]
    ultimo_flujo = bono[n]
    precio = -bono[0]
    fcid = 0 # inicializamos los flujos de caja intermedios descontados
    for t in range(1,n): # recorre los flujos de caja intermedios
        fcid += bono[t] / (1+etti[t-1])**t
    return (ultimo_flujo / (precio - fcid))**(1/n)-1

r02 = calcula_etti(bonoB, etti_calculada) # calculamos r02 conocido el bono B y r01
etti_calculada[1] = r02

r03 = calcula_etti(bonoC, etti_calculada) # calculamos r03 conocido el bono C, r01 y r02
etti_calculada[2] = r03

# Imprime la etti_calculada
for i in range(len(etti_calculada)):
    print(f"La ETTI del año {i+1} es {etti_calculada[i]:.2%}")

El precio del bono A es 909.09 €
El precio del bono B es 838.76 €
El precio del bono C es 1,484.52 €

La TIR del bono A es: 10.00%
La TIR del bono B es: 19.49%
La TIR del bono C es: 25.13%

La ETTI del año 1 es 10.00%
La ETTI del año 2 es 20.00%
La ETTI del año 3 es 30.00%


In [6]:
# Para calcular n
bonoA = [0,1000,0,0]          # n = 1, vector = [1]
bonoB = [0,90,1090,0]         # n = 2, vector = [1, 2]
bonoC = [0,500,500,1500]      # n = 3, vector = [1, 2, 3]
bono = bonoA
vector = [i for i, e in enumerate(bono) if e != 0]  # list comprhension
print(vector[-1])

1
